##Bag of Words
Target :
* metin listesi -> sayısal vektor

* sklearn countvectorizer: kelimelerin kaç defa geçtiğini sayar ve vektroe temsiline cevirir

sonuc:
* kelime kumesi
* her metin listesi sayısal vektorler ile temsil edilecek

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
dokumanlar = [
    "kedi bahçede",
    "kedi evde"
]

In [ ]:
#count vectorizer nesnesini oluştur
kelime_sayac = CountVectorizer()

In [ ]:
#dokumanlari sayisal vektorlere cevir(bag of words uygula)
dokuman_vektorleri = kelime_sayac.fit_transform(dokumanlar)
dokuman_vektorleri

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 4 stored elements and shape (2, 3)>

In [ ]:
#vocabulary olusturalım
kelime_kumesi = kelime_sayac.get_feature_names_out()
print(f"Kelime kumesi: {kelime_kumesi}")

Kelime kumesi: ['bahçede' 'evde' 'kedi']


In [ ]:
#vektor temsili
vektor_temsili = dokuman_vektorleri.toarray()
print(f"vektor temsili: {vektor_temsili}")

vektor temsili: [[1 0 1]
 [0 1 1]]


## Bag of Words
* IMDB film yorumları içeren veir seti ile bag of words

* text cleaning
* metinleri sayısal vektorlere cevirme
* kelime freekanslarını hesapla ve en sık gece n5 kelimeyi listele

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer #bag of words
import re #veri temizleme
from collections import Counter


In [ ]:
def load():
  data = pd.read_csv("IMDB Dataset.csv")
  return data

df = load()
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
yorumlar = df["review"]
yorumlar

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."
...,...
49995,I thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...


In [ ]:
etiketler = df["sentiment"]
etiketler

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
...,...
49995,positive
49996,negative
49997,negative
49998,negative


In [ ]:
#text cleaning
import nltk
import re
from nltk.corpus import stopwords

nltk.download("stopwords")
eng_stop_words = set(stopwords.words("english"))

eng_stop_words.add("movie")
eng_stop_words.add("film")


def metin_temizleme(metin):

  #tum harfleri kucuk harfe cevir
  metin = metin.lower()

  #rakamları kaldır
  metin = re.sub(r"\d+", "", metin)

  #ozel karakterleri kaldır
  metin = re.sub(r"[^\w\s]", "", metin)

  #2 karakterden kısa olan kelimeleri kaldır
  temiz_kelimeler = [
      kelime for kelime in metin.split()
      if kelime not in eng_stop_words and len(kelime) > 2
  ]

  return " ".join(temiz_kelimeler)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
temizlenmis_yorumlar = [metin_temizleme(y) for  y in yorumlar]

In [ ]:
#bow modeli eğitme
bow_modeli = CountVectorizer()


In [ ]:
#ilk 75 yorumu sayısal vektorlere çevirme
yorum_vektorleri = bow_modeli.fit_transform(temizlenmis_yorumlar[0:75])

In [ ]:
#vocab
kelime_kumesi = bow_modeli.get_feature_names_out()
kelime_kumesi

array(['abbot', 'abetted', 'abiding', ..., 'zone', 'zooms', 'zwick'],
      dtype=object)

In [ ]:
#her kelimenin toop. kaç adet geciği
kelime_sayilari = yorum_vektorleri.sum(axis = 0).A1

In [ ]:
#kelimeler ile frekansları bir sozlukte eşleştirleim
kelime_frekansi = dict(zip(kelime_kumesi, kelime_sayilari))

en_cok_gecen_5_kelime = Counter(kelime_frekansi).most_common(5)
en_cok_gecen_5_kelime

[('one', np.int64(72)),
 ('like', np.int64(59)),
 ('good', np.int64(38)),
 ('even', np.int64(37)),
 ('see', np.int64(36))]

##TF IDF


* Küçük bir belge oluştur
* TF-IDF vectorizer ile belgeleri sayısal vektörlere dönüştür
* Kelime kumesini olustur
* Belgelerin tf-idf vektor temsillerini elde edelim
* Tum belgeler icin kelimelerin ort.tf-idf değerlerini hesapla

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
belgeler = [
    "Köpek çok tatlı bir hayvandır",
    "Köpek ve kuşlar çok tatlı hayvanlardır",
    "İnekler süt üretirler"
]

In [ ]:
tfidf_modeli = TfidfVectorizer()

In [ ]:
#belgeleri sayısal vektörlere dönüştür
belge_vektorleri = tfidf_modeli.fit_transform(belgeler)

In [ ]:
#vocab oluştur
kelime_kumesi = tfidf_modeli.get_feature_names_out()


In [ ]:
#belgelerin tf idf değerlerini numpy formatına çevir
vektor_temsili = belge_vektorleri.toarray()


In [ ]:
vektor_temsili

array([[0.51741994, 0.51741994, 0.        , 0.        , 0.3935112 ,
        0.        , 0.        , 0.3935112 , 0.        , 0.3935112 ,
        0.        ],
       [0.        , 0.        , 0.45954803, 0.45954803, 0.34949812,
        0.        , 0.        , 0.34949812, 0.45954803, 0.34949812,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.57735027, 0.57735027, 0.        , 0.        , 0.        ,
        0.57735027]])

In [ ]:
'''
array([[0.51741994, 0.51741994, 0.        , 0.        , 0.3935112 ,0.        , 0.        , 0.3935112 , 0.        , 0.3935112 ,0.        ],
       [0.        , 0.        , 0.45954803, 0.45954803, 0.34949812,0.        , 0.        , 0.34949812, 0.45954803, 0.34949812,0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,0.57735027, 0.57735027, 0.        , 0.        , 0.        ,0.57735027]])
       '''



NameError: name 'array' is not defined

In [ ]:
#bunları okunabilir hale getir
df_tf_idf = pd.DataFrame(vektor_temsili, columns = kelime_kumesi)
df_tf_idf

,bir,hayvandır,hayvanlardır,kuşlar,köpek,nekler,süt,tatlı,ve,çok,üretirler
0,0.51742,0.51742,0.000000,0.000000,0.393511,0.00000,0.00000,0.393511,0.000000,0.393511,0.00000
1,0.00000,0.00000,0.459548,0.459548,0.349498,0.00000,0.00000,0.349498,0.459548,0.349498,0.00000
2,0.00000,0.00000,0.000000,0.000000,0.000000,0.57735,0.57735,0.000000,0.000000,0.000000,0.57735


In [ ]:
#her eklimenin belgeler arasındaki ort tf idf değerelerini hesapla
ort_tf_idf = df_tf_idf.mean(axis=0)
ort_tf_idf

,0
bir,0.172473
hayvandır,0.172473
hayvanlardır,0.153183
kuşlar,0.153183
köpek,0.247670
nekler,0.192450
süt,0.192450
tatlı,0.247670
ve,0.153183
çok,0.247670


In [ ]:
#ODEV
belgeler = df["review"]


In [ ]:
#test icin kucultuyorum
belgeler = belgeler[:5000]

In [ ]:
tfidf_modeli = TfidfVectorizer(
    stop_words = "english",
    max_features=5000,
    ngram_range=(1,1)
)

In [ ]:
belge_vektorleri = tfidf_modeli.fit_transform(belgeler)

In [ ]:
kelime_kumesi = tfidf_modeli.get_feature_names_out()

In [ ]:
vektor_temsili = belge_vektorleri.toarray()

In [ ]:
vektor_temsili

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.19869273, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
'''
array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,0.        ],
       [0.        , 0.19869273, 0.        , ..., 0.        , 0.        ,0.        ]])

'''

'\narray([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,0.        ],\n       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,0.        ],\n       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,0.        ],\n       ...,\n       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,0.        ],\n       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,0.        ],\n       [0.        , 0.19869273, 0.        , ..., 0.        , 0.        ,0.        ]])\n\n'

In [ ]:
df_tf_idf = pd.DataFrame(vektor_temsili, columns=kelime_kumesi)
df_tf_idf.head()

,000,10,100,11,12,13,13th,14,15,16,...,yesterday,york,young,younger,youth,zero,zoey,zombie,zombies,zone
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.082812,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
3,0.0,0.061198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.21961,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.099406,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0


In [ ]:
ort_tf_idf = df_tf_idf.mean(axis=0)
ort_tf_idf.sort_values(ascending=False).head(20)

,0
br,0.113989
movie,0.058989
film,0.049434
like,0.027560
just,0.026273
good,0.024874
story,0.020788
really,0.020493
time,0.020303
bad,0.019641
